In [2]:
#| label: imports-libraries
#| echo: false
#| output: false
#| include: true

## Data manipulation imports
import pandas as pd
import numpy as np

## Display imports
from IPython.display import display, Markdown

## Image extractor
from pypdf import PdfReader

## Plot imports
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (5,5/2.5)
import seaborn as sns
sns.set_style('whitegrid')
sns.set_theme()
sns.set_context(
    "paper", 
    rc={
        "figsize"       :   plt.rcParams['figure.figsize'],
        'font_scale'    :   1.25,
    }
)

def extract_images_from_pdf(
        output_path : str = "../imgs/",
        pdf_path : str = "../data/pdf/DC_airflow_chapter1.pdf",
        page_number : int = -5,
    ) -> None:
    ## Create an instance of PdfReader with the PDF path
    reader = PdfReader(pdf_path)
    ## Define the page to extract images from
    page = reader.pages[page_number]
    ## Create a counter to name the images
    count = 0
    try:
        ## Loop through the images in the page
        for image_file_object in page.images:
            with open(output_path + '/' + str(count) + image_file_object.name, "wb") as fp:
                print(image_file_object.name)
                fp.write(image_file_object.data)
                count += 1
        return None
    except:
        print("No images found in page")
        return None

# Introduction
This course notes corresponds to course [Docker Compose in Depth](https://github.com/avescodes/docker-compose-in-depth) on Udemy.

Docker has taken the development world by storm in recent years, being the first effective tool that wraps up a piece of software in a complete file system package, installs it on a server, and runs it repeatedly. However, until recently it was difficult to do this with micro-architectures composed of numerous containers that all need to work in conjunction with one another. Enter Docker Compose, the handiest tool to hit the tech world since Docker. Here’s everything you need to know...

- Define multi-container application environments
- Create flexible, customisable environments and networks
- Transform an existing application into a fully Docker-ised environment
- Enhance your Docker experience

## Make Your Docker Experience Even More Stress-Free

First you’ll cover the basic features using a sample environment, gaining an understanding of restarts, dependencies, and persisting the database with a volume.

After that you’ll progress to networks. You’ll take an in-depth look at isolating containers, aliases and container names, links, using external networks, and how updates affect networking. Then it’s on to the really good stuff; a section each is dedicated to volumes, logging, the Compose CLI, and ‘Composing Compose’ (don’t worry, it won’t be as complicated as it sounds by the time you get there). Finally you’ll learn about Compose in Production.

## About Docker Compose

Docker Compose is a useful tool from the people at Docker. It makes defining and running application environments made up of multiple Docker containers even easier and more efficient. Up until now, starting any more than one or two Docker containers was extremely complicated. With Docker Compose, the entire process just got infinitely better. 

# Docker Compose in Depth
Let us start right away and have a look at a Docker Compose file^[<https://gdevillele.github.io/compose/wordpress/>]. A Docker Compose file is essentially a `yaml` file, which defines the services that make up your app. Let us have a look at a simple example:

```yaml
version: '2'                    #<1>
services:                       #<2>  
  hello-world:                  #<3>
    image: hello-world          #<4>
    ports:                      #<5>
      - 8080:80                 #<6>
```

1. `version` sets the version and is required and all compose files
1. `services` maps container names and their configurations.
1. Defines a single container called `hello-world` 
1. Sets the `image` of the container to `tutum/hello-world`
1. `port` 80 is exposed

You'll notice this is very similar to the verbiage used in the Docker run command and in many cases commands in compose mirror those from docker run. To run this environment all you need to do is exit the file and get to a terminal in the same directory as Docker composed YAML. From there you can issue the command `docker-compose up -d` to make it run in detached mode to launch. Once running the previous command you will see something similar to this output, indicating the the images have been pulled, started and are running. 

```bash
[+] Running 2/2
 ✔ hello-world 1 layers [⣿]      0B/0B      Pulled                                                               3.1s 
   ✔ 70f5ac315c5a Pull complete                                                                                  0.6s 
[+] Running 1/1
 ✔ Container 0102-hello-world-1  Started                                                                         0.1s
```

Now if this is the first time you've run this you will see it pulling that image and then you should see creating network and creating the hello container to do that.
However we need to find out which port is being redirected from port 80. In order to do that, we use the command `docker ps` to list all the running containers.

```bash
docker ps 
```

and here we see it's port 8080
```bash
CONTAINER ID   IMAGE               COMMAND                  CREATED         STATUS         PORTS                  NAMES
20224eea69b5   tutum/hello-world   "/bin/sh -c 'php-fpm…"   3 minutes ago   Up 3 minutes   0.0.0.0:8080->80/tcp   0102-hello-world-1
```

Now we can use `curl` to see the output of the container. 
```bash
curl localhost:8080
```
Then, we can see the answer from the container. 
```html
<html>
<head>
        <title>Hello world!</title>
        <link href='http://fonts.googleapis.com/css?family=Open+Sans:400,700' rel='stylesheet' type='text/css'>
        <style>
        body {
                background-color: white;
                text-align: center;
                padding: 50px;
                font-family: "Open Sans","Helvetica Neue",Helvetica,Arial,sans-serif;
        }

        #logo {
                margin-bottom: 40px;
        }
        </style>
</head>
<body>
        <img id="logo" src="logo.png" />
        <h1>Hello world!</h1>
        <h3>My hostname is 20224eea69b5</h3>    </body>
</html>
```

# Docker Compose 
## Basics
Next, we are going to dive deeper into the basics of Docker Compose. We will start analyzing the [docker compose wordpress example](https://gdevillele.github.io/compose/wordpress/). This example contains the following information:


```yaml
version: '2'

services:
   db:                                  #<1>
     image: mysql:5.7                   #<1.1>
     volumes:                           #<1.2>
       - "./.data/db:/var/lib/mysql"
     restart: always                    #<1.3>
     environment:                       #<1.4>
       MYSQL_ROOT_PASSWORD: wordpress
       MYSQL_DATABASE: wordpress
       MYSQL_USER: wordpress
       MYSQL_PASSWORD: wordpress

   wordpress:                            #<2>
     depends_on:
       - db
     image: wordpress:4.5                #<2.1>
     links:                              #<2.2>
       - db
     ports:                              #<2.3>
       - "8000:80"
     restart: always
     environment:                        #<2.4>
       WORDPRESS_DB_HOST: db:3306
       WORDPRESS_DB_PASSWORD: wordpress
```

where:  

1. denotes the first service called `db`, 
    1. denotes the image used for the environment as a `mysql` database  
    1. creates a folder to persist the data in `./.data/db`
    1. restarts the container if it stops
    1. defines the environment variables for the database environment

1. denotes the second service called `wordpress`, which defines the `wordpress` environment  
    1. denotes the image used for the wordpress environment  
    1. denotes the links to the other service called `db`
    1. exposes port 80 of the container to port 8000 of the host
    1. defines the environment variables for the wordpress environment

Taking a look at the setup provided by docker-compose file, the following schema is obtained:

```{mermaid}
%%| label: docker_compose_example
%%| fig-cap: Schematics of the Docker compose example showing 
%%| fig-width: 8.0

flowchart LR
    subgraph Docker[Docker Container]
        direction TB

        subgraph Service1[Service: db]
            DB[("DB
            
            Persists data
            Restarts if it stops
            Defines environment variables
            Exposes port 3306")] 
            
        end
        
        subgraph Service2[Service: wordpress]
            Wordpress["Wordpress
            
            Depends on db
            Links to db
            Defines environment variables
            Exposes port 80"]
        end

        Service1 <--> Service2
        Service1 -- data is persisted in --> Volume[Volume]
    end

    subgraph Host
        direction LR
        Browser[Browser Port 8000]
    end

    Docker -- Maps port 80 to 8000 --> Host
    
```

Summarizing, we have review our first Docker compose file, which defines two services, a database and a wordpress environment. The database is linked to the wordpress environment and the wordpress environment is exposed to the host on port 8000.
Furthermore, the database is persisted in the host in the folder `./.data/db`. Next, we are going to dive deeper into the docker compose file and analyze how to persist data in a volume.

## Persiting Database with a Volume


```yaml
version: '2'

services:
  wordpress:
    image: wordpress:4.5
    depends_on:
      - db
    links:
      - db
    ports:
      - "8080:80"
    environment:
      WORDPRESS_DB_HOST: db:3306
      WORDPRESS_DB_USER: wordpress
      WORDPRESS_DB_PASSWORD: wordpress
    restart: always

  db:
    image: mariadb:10.1
    volumes:
      - "./volumes/db:/var/lib/mysql"       #<1>
    restart: always                         #<2>
    environment:
      MYSQL_ROOT_PASSWORD: example
      MYSQL_DATABASE: wordpress
      MYSQL_USER: wordpress
      MYSQL_PASSWORD: wordpress
```

1. Creates a folder to persist the data in `./volumes/db` (host path) to `/var/lib/mysql` (container path).
1. Restarts the container if it stops - more on this in the next part.

Let us now discuss restart policies as we've defined one of them for each of our containers (but we don't quite know exactly what that means yet). In the case of two separate containers each with their own restart policies you'll find that restarts occur independently: 

So for example: if the database container went down in this environment our restart policy would cause it to restart its own but independently of Wordpress would then be up and things might be running again. 

```{mermaid}
%%| label: independent_restarts
%%| fig-cap: Independent restarts

flowchart LR
    subgraph Docker["Case 1: without dependencies"]
        direction LR
        wordpress["wordpress"]
        db["db"]

        wordpress <-.-> db
        db -. restarts always .- db
    end
```

Many applications that depend on a data layer like this may just stop working if their database disappears and/or is replaced. Therefore, we define a hierarchy between containers. We can do that by using the `depends_on` keyword. So in this case we're going to say that the Wordpress container depends on the database container. 

```{mermaid}
%%| label: independent_restarts
%%| fig-cap: Restart using depends_on

flowchart LR
    subgraph Docker["Case 2: with depends_on"]
        direction LR
        wordpress["wordpress"]
        db["db"]

        wordpress -- depends on --> db
        db -. restarts always .- db
        wordpress -. restart in case db restarts .-> wordpress
    end
```

The `depends_on` does not propagate the in the other direction. In case the wordpress container goes down, the database container will not be restarted. 
